In [1]:
import struct
import numpy as np
import os
import time
import matplotlib.pyplot as plt
import datetime
import glob
import pandas as pd
from itertools import islice  
import xgboost as xgb
from model_config import *
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from scipy import stats
from sklearn.metrics import r2_score, explained_variance_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
#from treesbo.tuning import main_tuning_with_bo
%matplotlib inline
import matplotlib.pyplot as plt
import joblib

In [2]:
def lblrtm_tape11_reader(fname, opt):
    #%[波数，光学厚度]
    #% File format illustration
    #% for single precision
    #% shift 266*4 bytes
    #% LOOP
    #% 1 int        , 24 (block of v1, v2, dv, npts)
    #% 2 double vars, for v1, and v2
    #% 1 float      , for dv
    #% 1 int        , for npts
    #% 1 int        , 24
    #% 1 int        , 9600 or npts*4 (beg of block output)
    #% NPTs float   , rad
    #% 1 int        , 9600 or npts*4 (end of block of output)
    #% LOOP ENDS
    #
    #% for double precision
    #% shift 356*4 bytes
    #% LOOP
    #% 1 int        , 32 (v1, v2, dv and npts, extra 0)
    #% 3 double vars, for v1, v2, and dv
    #% 1 long int   , for npts
    #% 1 int        , 32   
    #% 1 int        , 19200 or npts*8 (beg of block of output)
    #% NPTS double  , rad
    #% 1 int        , 19200 or npts*8 (end of block of output)
    #% LOOP ENDS
    #%npts: the number of points in the panel?
    #% Author: Xianglei Huang
    #% Tested on Redhat Linux with pgi-compiler version of LBLRTM
    v = np.array([])
    rad = np.array([])
    if opt.lower() == 'float' or opt.lower() == 'single':
        shift = 266
        itype   = 1
    else:
        shift = 356
        itype = 2
    #print(shift,itype)
    fid = open(fname, 'rb')
    fid.seek(shift*4)
    #% decide whether need to open as big-endian file
    test = struct.unpack("i",fid.read(4))[0]  #matlab中的'int'对应c的'int'
    #print("test=",test)
    fid.close()
    if (itype == 1 and test == 24) or (itype ==2 and test == 32):
        fid = open(fname, 'rb')
        fid.seek(shift*4)
        order='<'
    else:
        fid = open(fname, 'rb')
        fid.seek(shift*4)
        order='>'        
    #print(order)
    endflg = 0
    panel = 0
#
    if itype == 1:   
        while endflg == 0:
            panel = panel + 1
            _ = struct.unpack(order+"i",fid.read(4))[0]
            v1= struct.unpack(order+"d",fid.read(8))[0]#matlab的'double'对应c的'double'，对应python的float，占8字节
            #print("v1=",v1)
            if np.isnan(v1):
                break
            v2= struct.unpack(order+"d",fid.read(8))[0]
            #print("v2=",v2)
            dv= struct.unpack(order+"f",fid.read(4))[0]#matlab的'float'对应c的'float'，对应python的float，占4字节
            #print("dv=",dv)
            npts= struct.unpack(order+"i",fid.read(4))[0]
            #print("npts=",npts)
            if npts!=2400:
                endflg=1
                #print("npts break"," and npts=",npts)
            _ = struct.unpack(order+"i",fid.read(4))[0]
            LEN= struct.unpack(order+"i",fid.read(4))[0]
            if LEN!=4*npts:
                #print('1 internal file inconsistency')
                endflg=1
            tmp= struct.unpack(order+str(npts)+"f",fid.read(4*npts))
            #print("tmp=",tmp)
            LEN2= struct.unpack(order+"i",fid.read(4))[0]
            if LEN != LEN2:
                #print('2 internal file inconsistency')
                endflg=1
            v=np.append(v,np.arange(v1,v2,dv).transpose())
            rad=np.append(rad,np.array(tmp).reshape(npts,1))
    else:
        print("itype != 1")
        while endflg == 0:
            panel = panel + 1
            _ = struct.unpack(order+"i",fid.read(4))[0]
            tmp= struct.unpack(order+"3d",fid.read(8*3))
            v1=tmp[0]
            v2=tmp[1]
            dv=tmp[2]
            if np.isnan(v1):
                break 
            npts= struct.unpack(order+"q",fid.read(8))[0] #npts = fread(fid, 1, 'int64');
            if npts!=2400:
                endflg=1
                #print("npts break")
            _ = struct.unpack(order+"i",fid.read(4))[0]   
            LEN= struct.unpack(order+"i",fid.read(4))[0]
            if LEN!=8*npts:
                #print('3 internal file inconsistency')
                endflg=1
            tmp= struct.unpack((order+str(npts)+"d"),fid.read(8*npts))
            LEN2= struct.unpack(order+"i",fid.read(4))[0]
            if LEN != LEN2:
                #print('4 internal file inconsistency')
                endflg=1
            v=np.append(v,np.arange(v1,v2,dv).transpose())
            rad=np.append(rad,np.array(tmp).reshape(npts,1))                
    fid.close()
    return v,rad

In [3]:
def EC(fname):
    p=np.array([])
    t=np.array([])
    wv=np.array([])
    co2=np.array([])
    o3=np.array([])
    no2=np.array([])
    co=np.array([])
    ch4=np.array([])
    input_file=open(fname)
    num=0
    for line in islice(input_file,19,None):
        num=num+1
        #print(line)
        p_temp,t_temp,wv_temp,co2_temp,o3_temp,no2_temp,co_temp,ch4_temp=[float(i) for i in line.split()]
        p=np.append(p,p_temp)
        t=np.append(t,t_temp)
        wv=np.append(wv,wv_temp)
        co2=np.append(co2,co2_temp)
        o3=np.append(o3,o3_temp)
        no2=np.append(no2,no2_temp)
        co=np.append(co,co_temp)
        ch4=np.append(ch4,ch4_temp)
        if num%101==0:
            for line2 in islice(input_file, 0, 2):
                continue
            
    return p,t,wv,co2,o3,no2,co,ch4

In [4]:
def func_evaluation(y_real, y_pred, verbose=True):
    """
    y_real: real values
    y_pred: prediction values
    MAKE THEM HAVE THE SHAPE OF (N,) FIRST BY USING ndarray.ravel()
    """
    if not isinstance(y_real, np.ndarray):
        y_real = np.array(y_real)
    if not isinstance(y_pred, np.ndarray):
        y_pred = np.array(y_pred)
    y_real = y_real.ravel()
    y_pred = y_pred.ravel()

    # MAPE
    def cal_mape(y_true, y_pred):
        y_true = np.where(y_true == 0, 1, y_true)  # To avoid zeros.
        mape = np.mean(abs((y_true.ravel() - y_pred.ravel()) / y_true.ravel()))
        return mape

    m0 = stats.pearsonr(y_real, y_pred)[0]
    m1 = r2_score(y_real, y_pred)
    m2 = explained_variance_score(y_real, y_pred)
    m3 = mean_squared_error(y_real, y_pred)
    m4 = mean_absolute_error(y_real, y_pred)
    m5 = cal_mape(y_real, y_pred)
    if verbose:
        print("模型的评估结果:")
        print(
            'PersonR: %f \nRMSE: %f \nR2: %f \nExplained Variance: %f \nMSE: %f \nMAE: %f \nMAPE: %f'
            % (m0, np.sqrt(m3), m1, m2, m3, m4, m5))
    return m0, np.sqrt(m3), m1, m2, m3, m4, m5



In [5]:
ecpath='/mnt/FY4_GIIRS_OD'
ozopath='/mnt/FY4_GIIRS_OD/ozo/angle1'
ec_name=ecpath+'/ECMWF_83P_101L.dat'
#print(type(test_name))
p,t,wv,co2,o3,no2,co,ch4=EC(ec_name)   
#print(t)
ec_input=np.hstack((p.reshape(-1,1),t.reshape(-1,1),wv.reshape(-1,1),co2.reshape(-1,1),o3.reshape(-1,1),no2.reshape(-1,1),co.reshape(-1,1),ch4.reshape(-1,1)))
print(ec_input.shape)
ec_input_100=np.zeros((83*100,ec_input.shape[1]))
for pro in range(0,83): #83条
    for i in range(0+101*pro,100+101*pro):
        for j in range(ec_input.shape[1]):
            ec_input_100[i-101*pro+100*pro,j]=(ec_input[i,j]+ec_input[i+1,j])/2.0
print(ec_input_100.shape)

(8383, 8)
(8300, 8)


In [ ]:
#把EC83的廓线按照第一条，第一层复制55万个，然后第二条、第一层，共83*100*55万个数据。
ec_df=pd.DataFrame(ec_input_100)
#print(ec_df)
true_total=np.array([])
pre_total=np.array([])

for pro in range(0,83):
    #tot=np.array([])
    chara1=np.array([])
    chara2=np.array([])
    v=np.array([])
    mg_od=np.array([])
    ec_new=np.array([])
    for i in range(99,-1,-1):
    #f = open("metrics_%d.txt" %(i), "w")
    #   for pro in range(0,83):
     #倒叙原因是为与ozo文件统一，ozo中第一层是最底层
        ec_new_temp=ec_df.iloc[i+pro*100].values.reshape(1,-1)   #shape=(1,8)
        #print("ec_new_temp",ec_new_temp.shape)
        #new=np.repeat(ec_df.iloc[i+pro*100].values.reshape(1,-1),555001,axis=0)
        #print(new.shape)
        #tot=np.append(tot,new)
        lev=100-i
        #####开始处理ozo的小文件
        fix_name = glob.glob(ozopath+'/*_%dP_*%03d' %(pro+1,lev))
        if os.path.exists(fix_name[0]):
            v_temp,mg_od_temp = lblrtm_tape11_reader(fix_name[0],'single')
            mg_od_temp=np.exp(-mg_od_temp)
            #print("小文件测试",v_temp.shape,mg_od_temp.shape)
            #print(np.where(mg_od_temp==np.max(mg_od_temp)))
            #print(v_temp[22515])
            #fig = plt.figure(figsize=(15,10))
            #ax1 = fig.add_subplot(211)         
            #l1,=ax1.plot(v_temp,mg_od_temp,'g-',linewidth=1.5)
            #ax2 = fig.add_subplot(212)
            #l2,=ax2.plot(mg_od_temp,'r:', linewidth=1.5)
            #plt.show()            
        else:
            v_temp=np.full([555001,], np.nan)
            mg_od_temp=np.full([555001,], np.nan)
        v=np.append(v,v_temp) ###每个小文件有555001个数据，shape=555001*100
        mg_od=np.append(mg_od,mg_od_temp)
        ec_new=np.append(ec_new,ec_new_temp)# shape=(100,8)
    print(v.shape,mg_od.shape,ec_new.shape)
    v=v.reshape(100,-1)
    mg_od=mg_od.reshape(100,-1)
    ec_new=ec_new.reshape(100,-1)
    print(v.shape,mg_od.shape,ec_new.shape)
    ###开始计算某个波数段的垂直层次的偏导数
    #mg_od_1=np.zeros((mg_od.shape[0],mg_od.shape[1]))
    #ec_new_1=np.zeros((ec_new.shape[0],ec_new.shape[1]))
    #for i in range(0,100): #赋值一个新的数组
    #    if i==99:
    #        mg_od_1[i]=mg_od[i]
    #        ec_new_1[i]=ec_new[i]
    #    else:
    #        mg_od_1[i]=mg_od[i+1]
    #        ec_new_1[i]=ec_new[i+1]
    #print("ec_new_1",ec_new_1.shape)        
    for i in range(0,100): #最后一层不做偏导
        if i==99:
            chara1_temp=(mg_od[i])/(np.log(ec_new[i,0]))
            chara2_temp=np.log(mg_od[i]) 
        else:
            #tmp=np.array(list(map(lambda x: -0.15*8**2*x,tk_p)))
            chara1_temp=(mg_od[i+1]-mg_od[i])/(np.log(ec_new[i+1,0])-np.log(ec_new[i,0]))
            chara2_temp=np.log(mg_od[i+1]/mg_od[i])
        #print("chara1_temp",chara1_temp.shape)
        chara1=np.append(chara1,chara1_temp) #每个层都有555001个波数，chara1共应该有555001*100个数
        chara2=np.append(chara2,chara2_temp)
        #print("chara1_临时",chara1.shape)
    print("chara1_final",chara1.shape)#shape=55500100
    
    for i in range(0,100): #开始分层写入文件
        pd.DataFrame(chara1[i*555001:(i+1)*555001]).to_csv('./chara/chara1_%dP_%03d.csv'%(pro+1,i))
        pd.DataFrame(chara2[i*555001:(i+1)*555001]).to_csv('./chara/chara2_%dP_%03d.csv'%(pro+1,i))
    #if pro==0:
    #    break



(55500100,) (55500100,) (800,)
(100, 555001) (100, 555001) (100, 8)


/home/liuli/anaconda3/envs/JT/lib/python3.7/site-packages/ipykernel_launcher.py:67: RuntimeWarning: divide by zero encountered in true_divide
/home/liuli/anaconda3/envs/JT/lib/python3.7/site-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in true_divide
/home/liuli/anaconda3/envs/JT/lib/python3.7/site-packages/ipykernel_launcher.py:67: RuntimeWarning: divide by zero encountered in log


chara1_final (55500100,)
(55500100,) (55500100,) (800,)
(100, 555001) (100, 555001) (100, 8)
chara1_final (55500100,)
(55500100,) (55500100,) (800,)
(100, 555001) (100, 555001) (100, 8)
chara1_final (55500100,)
(55500100,) (55500100,) (800,)
(100, 555001) (100, 555001) (100, 8)
chara1_final (55500100,)
(55500100,) (55500100,) (800,)
(100, 555001) (100, 555001) (100, 8)
chara1_final (55500100,)
(55500100,) (55500100,) (800,)
(100, 555001) (100, 555001) (100, 8)
chara1_final (55500100,)
(55500100,) (55500100,) (800,)
(100, 555001) (100, 555001) (100, 8)
chara1_final (55500100,)
(55500100,) (55500100,) (800,)
(100, 555001) (100, 555001) (100, 8)
chara1_final (55500100,)
